In [1]:
from mnist_data import *
from mnist_classifier import *
from cifar_data import *
from cifar_classifier import *

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
USE_FEATURES = True
DATASET_TYPE = Cifar10Data
CLASSIFIER_TYPE = Cifar10Classifier
CLASS_REMOVED = 3
EPOCHS = 3

cls_generator = CLASSIFIER_TYPE(use_features = USE_FEATURES, epochs = EPOCHS)

In [3]:
d_generator = DATASET_TYPE(use_features=USE_FEATURES, class_removed=CLASS_REMOVED)

# assert len(np.unique(np.argmax(d.into_fit()[1], axis=1))) == 10
# assert len(np.unique(np.argmax(d.into_fit()[3], axis=1))) == 10

In [4]:
d_generator.set_removed_class(CLASS_REMOVED)

# assert len(np.unique(np.argmax(d.into_fit()[1], axis=1))) == 9
# assert len(np.unique(np.argmax(d.into_fit()[3], axis=1))) == 9


Removed class # 3
current number of examples per class -- train:
 {0: 5000, 1: 5000, 2: 5000, 4: 5000, 5: 5000, 6: 5000, 7: 5000, 8: 5000, 9: 5000}

current number of examples per class -- test:
 {0: 893, 1: 904, 2: 900, 4: 909, 5: 898, 6: 909, 7: 891, 8: 896, 9: 887}


In [5]:
cls_generator.fit(*d_generator.into_fit())

Train on 45000 samples, validate on 8087 samples
Epoch 1/3
45000/45000 [==============================] - 2s 51us/step - loss: 0.2350 - acc: 0.9637 - val_loss: 0.3119 - val_acc: 0.8966
Epoch 2/3
45000/45000 [==============================] - 2s 34us/step - loss: 0.0386 - acc: 0.9950 - val_loss: 0.3288 - val_acc: 0.8984
Epoch 3/3
45000/45000 [==============================] - 1s 33us/step - loss: 0.0193 - acc: 0.9983 - val_loss: 0.3519 - val_acc: 0.8979


In [6]:
cls_generator.set_trainability(is_trainable=False)

Classifier was set to NOT trainable!


In [7]:
from generator import *
generator = LowShotGenerator(cls_generator.model,
                             d_generator,
                             epochs=2,
                             n_clusters=30,
                             λ=0.5,
                             hidden_size=256)

Fitting generator
Epoch 1/2
62640/62640 [==============================] - 6s 94us/step - loss: 0.0313 - generator_loss: 0.0247 - classifier_loss: 0.0379 - generator_acc: 0.0079 - classifier_acc: 0.9902
Epoch 2/2
62640/62640 [==============================] - 6s 91us/step - loss: 0.0091 - generator_loss: 0.0170 - classifier_loss: 0.0013 - generator_acc: 0.0468 - classifier_acc: 1.0000


In [8]:
# n_samples = d.get_n_samples(1)

In [9]:
# generated_data, triplets = generator.generate_from_samples(n_samples, n_total=20, return_triplets= True)

In [10]:
# import matplotlib.pyplot as plt

# def add_image_to_figure(vector, width=28, height=28):
#     fig = plt.imshow(np.reshape(vector, (height, width)), interpolation='nearest')
#     fig.axes.get_xaxis().set_visible(False)
#     fig.axes.get_yaxis().set_visible(False)

In [11]:
# I = 2

In [12]:
# add_image_to_figure(generated_data[I], width=64, height=32)

In [13]:
# add_image_to_figure(n_samples[0], width=64, height=32)

In [14]:
# add_image_to_figure(triplets[I][2], width=64, height=32)

In [15]:
# add_image_to_figure(triplets[I][1], width=64, height=32)

In [16]:
all_d = DATASET_TYPE(use_features = USE_FEATURES, class_removed = CLASS_REMOVED)
all_cls = CLASSIFIER_TYPE(use_features = USE_FEATURES, epochs = EPOCHS)

In [17]:
all_cls.fit(*all_d.into_fit())

Train on 50000 samples, validate on 9000 samples
Epoch 1/3
50000/50000 [==============================] - 2s 36us/step - loss: 0.3839 - acc: 0.9001 - val_loss: 0.4468 - val_acc: 0.8446
Epoch 2/3
50000/50000 [==============================] - 2s 32us/step - loss: 0.1719 - acc: 0.9437 - val_loss: 0.4540 - val_acc: 0.8449
Epoch 3/3
50000/50000 [==============================] - 2s 32us/step - loss: 0.1408 - acc: 0.9528 - val_loss: 0.4661 - val_acc: 0.8454


In [18]:
all_cls.evaluate(*all_d.into_evaluate())

9000/9000 [==============================] - 0s 53us/step
Test loss: 0.46610726385646395
Test accuracy: 0.8454444444444444


[0.46610726385646395, 0.8454444444444444]

In [19]:
# for i in range(10):
#     samples = all_d.into_evaluate_one_class(class_index=i)[0]
#     all_cls.evaluate(samples, all_d._one_hot_encode(np.repeat(i, len(samples))))

# IS GENERATED DATA IN THE CORRECT CLASS

In [20]:
in_class_mask = all_d.y_dev == CLASS_REMOVED
all_samples = all_d.x_dev[in_class_mask]

samples = all_samples[:5]
all_cls.evaluate(samples, all_d._one_hot_encode(np.repeat(CLASS_REMOVED, len(samples))))


generated_data = generator.generate_from_samples(samples, n_total=1000)
all_cls.evaluate(generated_data, all_d._one_hot_encode(np.repeat(CLASS_REMOVED, len(generated_data))))

5/5 [==============================] - 0s 312us/step
Test loss: 1.3896974325180054
Test accuracy: 0.4000000059604645
Selecting category randomally
Selecting centroids randomally
5/5 [==============================] - 0s 290us/step
Test loss: 1.6103862524032593
Test accuracy: 0.0


[1.6103862524032593, 0.0]

In [21]:
FIX = True

d = DATASET_TYPE(use_features=USE_FEATURES, class_removed=CLASS_REMOVED)
cls = CLASSIFIER_TYPE(use_features = USE_FEATURES, epochs = EPOCHS)



d.set_removed_class(CLASS_REMOVED)

d.set_generated_data(np.concatenate((samples, generated_data)))



test_unique, test_counts = np.unique(d._one_hot_decode(d.into_fit(fix_class_imbalance = FIX)[1]), return_counts=True)
print("TRAIN:")
print(dict(zip(test_unique, test_counts)))

test_unique, test_counts = np.unique(d._one_hot_decode(d.into_fit(fix_class_imbalance = FIX)[3]), return_counts=True)
print("TEST:")
print(dict(zip(test_unique, test_counts)))







Removed class # 3
current number of examples per class -- train:
 {0: 5000, 1: 5000, 2: 5000, 4: 5000, 5: 5000, 6: 5000, 7: 5000, 8: 5000, 9: 5000}

current number of examples per class -- test:
 {0: 893, 1: 904, 2: 900, 4: 909, 5: 898, 6: 909, 7: 891, 8: 896, 9: 887}
TRAIN:
{0: 5001, 1: 5001, 2: 5001, 3: 5001, 4: 5001, 5: 5001, 6: 5001, 7: 5001, 8: 5001, 9: 5001}
TEST:
{0: 893, 1: 904, 2: 900, 3: 913, 4: 909, 5: 898, 6: 909, 7: 891, 8: 896, 9: 887}


In [22]:
test_unique, test_counts = np.unique(d._one_hot_decode(d.into_evaluate()[1]), return_counts=True)
print("BBBBB")
print(dict(zip(test_unique, test_counts)))

BBBBB
{0: 893, 1: 904, 2: 900, 3: 913, 4: 909, 5: 898, 6: 909, 7: 891, 8: 896, 9: 887}


In [23]:
cls.fit(*d.into_fit(fix_class_imbalance = FIX))
# cls.predict(d.into_evaluate_one_class(CLASS_REMOVED)[0])

Train on 50010 samples, validate on 9000 samples
Epoch 1/3
50010/50010 [==============================] - 2s 36us/step - loss: 0.2370 - acc: 0.9656 - val_loss: 1.0670 - val_acc: 0.8044
Epoch 2/3
50010/50010 [==============================] - 2s 32us/step - loss: 0.0277 - acc: 0.9972 - val_loss: 1.3599 - val_acc: 0.8054
Epoch 3/3
50010/50010 [==============================] - 2s 33us/step - loss: 0.0119 - acc: 0.9993 - val_loss: 1.5933 - val_acc: 0.8066


In [24]:
cls.evaluate(samples, d._one_hot_encode(np.repeat(CLASS_REMOVED,len(samples))))
cls.evaluate(generated_data, d._one_hot_encode(np.repeat(CLASS_REMOVED,len(generated_data))))

5/5 [==============================] - 0s 361us/step
Test loss: 5.3371732064988464e-05
Test accuracy: 1.0
5/5 [==============================] - 0s 292us/step
Test loss: 0.00016177052748389542
Test accuracy: 1.0


[0.00016177052748389542, 1.0]

# RESULTS WITH GENERATED:

In [25]:
cls.evaluate(*d.into_evaluate_one_class(CLASS_REMOVED))

913/913 [==============================] - 0s 49us/step
Test loss: 12.432678926683073
Test accuracy: 0.001095290251916758


[12.432678926683073, 0.001095290251916758]

In [26]:
len(samples)
len(generated_data)


5

In [27]:
d.set_generated_data(samples)

test_unique, test_counts = np.unique(d._one_hot_decode(d.into_fit(fix_class_imbalance = FIX)[1]), return_counts=True)
print("AAAAA")
print(dict(zip(test_unique, test_counts)))

test_unique, test_counts = np.unique(d._one_hot_decode(d.into_fit(fix_class_imbalance = FIX)[3]), return_counts=True)
print("BBBBB")
print(dict(zip(test_unique, test_counts)))


cls.fit(*d.into_fit(fix_class_imbalance = FIX))
# cls.evaluate(*d.into_evaluate_one_class(CLASS_REMOVED))

AAAAA
{0: 5001, 1: 5001, 2: 5001, 3: 5001, 4: 5001, 5: 5001, 6: 5001, 7: 5001, 8: 5001, 9: 5001}
BBBBB
{0: 893, 1: 904, 2: 900, 3: 913, 4: 909, 5: 898, 6: 909, 7: 891, 8: 896, 9: 887}
Train on 50010 samples, validate on 9000 samples
Epoch 1/3
50010/50010 [==============================] - 2s 35us/step - loss: 0.2126 - acc: 0.9694 - val_loss: 1.2402 - val_acc: 0.8039
Epoch 2/3
50010/50010 [==============================] - 2s 32us/step - loss: 0.0277 - acc: 0.9972 - val_loss: 1.5701 - val_acc: 0.8058
Epoch 3/3
50010/50010 [==============================] - 2s 32us/step - loss: 0.0121 - acc: 0.9993 - val_loss: 1.7464 - val_acc: 0.8060


In [28]:
cls.evaluate(samples, d._one_hot_encode(np.repeat(CLASS_REMOVED,len(samples))))

5/5 [==============================] - 0s 286us/step
Test loss: 1.394749915561988e-06
Test accuracy: 1.0


[1.394749915561988e-06, 1.0]

In [29]:
cls.evaluate(generated_data, d._one_hot_encode(np.repeat(CLASS_REMOVED,len(generated_data))))

5/5 [==============================] - 0s 322us/step
Test loss: 2.9003474712371826
Test accuracy: 0.0


[2.9003474712371826, 0.0]

# RESULTS WITH SAMPLES ONLY

In [30]:
cls.evaluate(*d.into_evaluate_one_class(CLASS_REMOVED))

913/913 [==============================] - 0s 53us/step
Test loss: 13.956647715197727
Test accuracy: 0.0


[13.956647715197727, 0.0]

In [32]:
all_cls.evaluate(*d.into_evaluate_one_class(CLASS_REMOVED))

913/913 [==============================] - 0s 51us/step
Test loss: 0.7902770680900733
Test accuracy: 0.6900328590339795


[0.7902770680900733, 0.6900328590339795]